# Benchmark Dataset을 통한 Nbeats 모델 성능 검증

## Install packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install -U kaleido
! pip install psutil
! pip install utils
! conda install -c plotly plotly-orca psutil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: conda: command not found


In [ ]:
%matplotlib inline
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import optim
from torch.nn import functional as F
import os

# from nbeats_pytorch.model import NBeatsNet
# from trainer_pytorch import save

warnings.filterwarnings(action='ignore', message='Setting attributes')
from progressbar import *
import plotly.express as px
import plotly.io as pio
import kaleido
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.stats import norm
from scipy.stats import t

## Nbeats model

In [ ]:
# Model Class
import pickle
import random
import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.optim import Optimizer

class NBeatsNet(nn.Module):
    SEASONALITY_BLOCK = 'seasonality'
    TREND_BLOCK = 'trend'
    GENERIC_BLOCK = 'generic'

    def __init__(self,
                 device=torch.device('cpu'),
                 stack_types=(TREND_BLOCK, SEASONALITY_BLOCK),
                 nb_blocks_per_stack=3,
                 forecast_length=5,
                 backcast_length=10,
                 thetas_dim=(4, 8),
                 share_weights_in_stack=False,
                 hidden_layer_units=256,
                 nb_harmonics=None,
                 dropout_rate=0.3):
        super(NBeatsNet, self).__init__()
        self.forecast_length = forecast_length
        self.backcast_length = backcast_length
        self.hidden_layer_units = hidden_layer_units
        self.nb_blocks_per_stack = nb_blocks_per_stack
        self.share_weights_in_stack = share_weights_in_stack
        self.nb_harmonics = nb_harmonics
        self.dropout_rate = dropout_rate
        self.stack_types = stack_types
        self.stacks = []
        self.thetas_dim = thetas_dim
        self.parameters = []
        self.device = device
        print('| N-Beats')
        for stack_id in range(len(self.stack_types)):
            self.stacks.append(self.create_stack(stack_id))
        self.parameters = nn.ParameterList(self.parameters)
        self.to(self.device)
        self._loss = None
        self._opt = None

    def create_stack(self, stack_id):
        stack_type = self.stack_types[stack_id]
        print(f'| --  Stack {stack_type.title()} (#{stack_id}) (share_weights_in_stack={self.share_weights_in_stack})')
        blocks = []
        for block_id in range(self.nb_blocks_per_stack):
            block_init = NBeatsNet.select_block(stack_type)
            if self.share_weights_in_stack and block_id != 0:
                block = blocks[-1]  # pick up the last one when we share weights.
            else:
                block = block_init(self.hidden_layer_units, self.thetas_dim[stack_id],
                                   self.device, self.backcast_length, self.forecast_length, self.nb_harmonics)
                self.parameters.extend(block.parameters())
            print(f'     | -- {block}')
            blocks.append(block)
        return blocks

    def save(self, filename: str):
        torch.save(self, filename)

    @staticmethod
    def load(f, map_location=None, pickle_module=pickle, **pickle_load_args):
        return torch.load(f, map_location, pickle_module, **pickle_load_args)

    @staticmethod
    def select_block(block_type):
        if block_type == NBeatsNet.SEASONALITY_BLOCK:
            return SeasonalityBlock
        elif block_type == NBeatsNet.TREND_BLOCK:
            return TrendBlock
        else:
            return GenericBlock


    def forward(self, backcast):
        backcast = squeeze_last_dim(backcast)
        forecast = torch.zeros(size=(backcast.size()[0], self.forecast_length,))  # maybe batch size here.
        
        trend_forecast = torch.zeros(size=(backcast.size()[0], self.forecast_length,))
        seasonal_forecast = torch.zeros(size=(backcast.size()[0], self.forecast_length,))

        for stack_id in range(len(self.stacks)):
            for block_id in range(len(self.stacks[stack_id])):
                b, f = self.stacks[stack_id][block_id](backcast)
                backcast = backcast.to(self.device) - b
                forecast = forecast.to(self.device) + f
                if type(self.stacks[stack_id][block_id]).__name__ == 'TrendBlock':
                    trend_forecast = trend_forecast.to(self.device) + f
                elif type(self.stacks[stack_id][block_id]).__name__ == 'SeasonalityBlock':
                    seasonal_forecast = seasonal_forecast.to(self.device) + f

        return backcast, forecast, trend_forecast, seasonal_forecast


def squeeze_last_dim(tensor):
    if len(tensor.shape) == 3 and tensor.shape[-1] == 1:  # (128, 10, 1) => (128, 10).
        return tensor[..., 0]
    return tensor


def seasonality_model(thetas, t, device):
    p = thetas.size()[-1]
    assert p <= thetas.shape[1], 'thetas_dim is too big.'
    p1, p2 = (p // 2, p // 2) if p % 2 == 0 else (p // 2, p // 2 + 1)
    s1 = torch.tensor([np.cos(2 * np.pi * i * t) for i in range(p1)]).float()  # H/2-1
    s2 = torch.tensor([np.sin(2 * np.pi * i * t) for i in range(p2)]).float()
    S = torch.cat([s1, s2])
    return thetas.mm(S.to(device))


def trend_model(thetas, t, device):
    p = thetas.size()[-1]
    assert p <= 4, 'thetas_dim is too big.'
    T = torch.tensor([t ** i for i in range(p)]).float()
    return thetas.mm(T.to(device))


def linear_space(backcast_length, forecast_length, is_forecast=True):
    horizon = forecast_length if is_forecast else backcast_length
    return np.arange(0, horizon) / horizon


class Block(nn.Module):

    def __init__(self, units, thetas_dim, device, backcast_length=10, forecast_length=5, share_thetas=False,
                 nb_harmonics=None, dropout_rate=0.2):
        super(Block, self).__init__()
        self.units = units
        self.thetas_dim = thetas_dim
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        self.share_thetas = share_thetas
        self.fc1 = nn.Linear(backcast_length, units)
        self.fc2 = nn.Linear(units, units)
        self.fc3 = nn.Linear(units, units)
        self.fc4 = nn.Linear(units, units)
        self.dropout = nn.Dropout(dropout_rate)  # dropout 추가 => 랜덤성 부여 효과를 볼 수 있음.  
        self.device = device
        self.backcast_linspace = linear_space(backcast_length, forecast_length, is_forecast=False)
        self.forecast_linspace = linear_space(backcast_length, forecast_length, is_forecast=True)
        if share_thetas:
            self.theta_f_fc = self.theta_b_fc = nn.Linear(units, thetas_dim, bias=False)
        else:
            self.theta_b_fc = nn.Linear(units, thetas_dim, bias=False)
            self.theta_f_fc = nn.Linear(units, thetas_dim, bias=False)

    def forward(self, x):
        x = squeeze_last_dim(x)
        x = F.relu(self.fc1(x.to(self.device))) 
        x = F.relu(self.fc2(x))
        x = self.dropout(x)  # add dropout 
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        return x

    def __str__(self):
        block_type = type(self).__name__
        return f'{block_type}(units={self.units}, thetas_dim={self.thetas_dim}, ' \
               f'backcast_length={self.backcast_length}, forecast_length={self.forecast_length}, ' \
               f'share_thetas={self.share_thetas}) at @{id(self)}'


class SeasonalityBlock(Block):

    def __init__(self, units, thetas_dim, device, backcast_length=10, forecast_length=5, nb_harmonics=None):
        if nb_harmonics:
            super(SeasonalityBlock, self).__init__(units, nb_harmonics, device, backcast_length,
                                                   forecast_length, share_thetas=True)
        else:
            super(SeasonalityBlock, self).__init__(units, forecast_length, device, backcast_length,
                                                   forecast_length, share_thetas=True)

    def forward(self, x):
        x = super(SeasonalityBlock, self).forward(x)
        backcast = seasonality_model(self.theta_b_fc(x), self.backcast_linspace, self.device)
        forecast = seasonality_model(self.theta_f_fc(x), self.forecast_linspace, self.device)
        return backcast, forecast


class TrendBlock(Block):

    def __init__(self, units, thetas_dim, device, backcast_length=10, forecast_length=5, nb_harmonics=None):
        super(TrendBlock, self).__init__(units, thetas_dim, device, backcast_length,
                                         forecast_length, share_thetas=True)

    def forward(self, x):
        x = super(TrendBlock, self).forward(x)
        backcast = trend_model(self.theta_b_fc(x), self.backcast_linspace, self.device)
        forecast = trend_model(self.theta_f_fc(x), self.forecast_linspace, self.device)
        return backcast, forecast


class GenericBlock(Block):

    def __init__(self, units, thetas_dim, device, backcast_length=10, forecast_length=5, nb_harmonics=None):
        super(GenericBlock, self).__init__(units, thetas_dim, device, backcast_length, forecast_length)

        self.backcast_fc = nn.Linear(thetas_dim, backcast_length)
        self.forecast_fc = nn.Linear(thetas_dim, forecast_length)

    def forward(self, x):
        # no constraint for generic arch.
        x = super(GenericBlock, self).forward(x)

        theta_b = self.theta_b_fc(x)
        theta_f = self.theta_f_fc(x)

        backcast = self.backcast_fc(theta_b)  # generic. 3.3.
        forecast = self.forecast_fc(theta_f)  # generic. 3.3.

        return backcast, forecast

# Birth Data set

## Data Preprocessing

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/2022 컨퍼/benchmark_data/benchmark_Births.csv', header=0)
data = data[['Date', 'Births']]
data.set_index('Date', inplace=True)
data2 = data.copy()
data2

,Births
Date,
1959-01-01,35
1959-01-02,32
1959-01-03,30
1959-01-04,31
1959-01-05,44
...,...
1959-12-27,37
1959-12-28,52
1959-12-29,48


## Load Scaler & Model

In [ ]:
window_size = 30
yscaler = pickle.load(open('/content/drive/Shareddrives/[투빅스] 16기 & 17기 시계열 컨퍼런스/예측/Nbeats_benchmark/Births/scaler.pkl', 'rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator StandardScaler from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [ ]:
model = NBeatsNet(stack_types=(NBeatsNet.TREND_BLOCK, NBeatsNet.SEASONALITY_BLOCK),
                    forecast_length=15,
                    backcast_length=30,
                    hidden_layer_units=128,
                    )
model.load_state_dict(torch.load('/content/drive/Shareddrives/[투빅스] 16기 & 17기 시계열 컨퍼런스/예측/Nbeats_benchmark/Births/model.pth', map_location='cpu'))

| N-Beats
| --  Stack Trend (#0) (share_weights_in_stack=False)
     | -- TrendBlock(units=128, thetas_dim=4, backcast_length=30, forecast_length=15, share_thetas=True) at @140655480700304
     | -- TrendBlock(units=128, thetas_dim=4, backcast_length=30, forecast_length=15, share_thetas=True) at @140655480758352
     | -- TrendBlock(units=128, thetas_dim=4, backcast_length=30, forecast_length=15, share_thetas=True) at @140655480697296
| --  Stack Seasonality (#1) (share_weights_in_stack=False)
     | -- SeasonalityBlock(units=128, thetas_dim=15, backcast_length=30, forecast_length=15, share_thetas=True) at @140655480759376
     | -- SeasonalityBlock(units=128, thetas_dim=15, backcast_length=30, forecast_length=15, share_thetas=True) at @140655486651344
     | -- SeasonalityBlock(units=128, thetas_dim=15, backcast_length=30, forecast_length=15, share_thetas=True) at @140655486649872


<All keys matched successfully>

### Predict

In [ ]:
def main(sample_size, prediction_size, yscaler):
    y_pred = np.zeros((sample_size, prediction_size))
    trend = np.zeros((sample_size, prediction_size))
    seasonality = np.zeros((sample_size, prediction_size))
    for i in range(sample_size):
        backcast, pred, trend_forecast, seasonal_forecast = model(Y)
        pred = pred.detach().cpu().numpy()
        trend_forecast = trend_forecast.detach().cpu().numpy()
        seasonal_forecast = seasonal_forecast.detach().cpu().numpy()

        y_pred[i,:] = yscaler.inverse_transform(pred)
        trend[i,:] = yscaler.inverse_transform(trend_forecast)
        seasonality[i,:] = y_pred[i,:] - trend[i,:]
    return y_pred, trend, seasonality

In [ ]:
## 신뢰구간 구축
def confidence_interval(sample, alpha = 0.05):
    mean = np.mean(sample, axis=0)
    std_error = np.std(sample, axis=0)

    max_interval = mean + norm.ppf(alpha/2, loc = 0, scale = 1) * std_error/np.sqrt(len(sample))
    min_interval = mean - norm.ppf(alpha/2, loc = 0, scale = 1) * std_error/np.sqrt(len(sample))
    
    return mean, min_interval, max_interval

In [ ]:
def plot_prediction(y_pred, trend, seasonality, start_list):
    fig = make_subplots(
        subplot_titles=['True Vs Predicted','Trend','Seasonality'],
        rows=2, cols=2,
        vertical_spacing=0.15,
        horizontal_spacing=0.025,
        column_widths=[0.9, 0.6],
        row_heights=[0.8, 0.8],
        specs=[[{"rowspan": 2}, {}], [None, {}]])
            
    # confidence interval
    pred_mean, pred_min_interval, pred_max_interval = confidence_interval(y_pred, 0.01)
    trend_mean, trend_min_interval, trend_max_interval = confidence_interval(trend, 0.01)
    seasonal_mean, seasonal_min_interval, seasonal_max_interval = confidence_interval(seasonality, 0.01)
            
    # plot(1,1) - prediction vs real_value
    ## CI
    fig.add_trace(go.Scatter(name = 'Lower Bound',
                             x = pd.period_range(start = data_tmp['Births'].index[-1], end = None, periods = 16, freq = 'D').to_timestamp(),
                             y = np.concatenate([np.array(data.iloc[-start_num:, :]).flatten()[29].reshape(-1), pred_max_interval]),
                             mode = 'lines',
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             showlegend = False), row=1,col=1)
    fig.add_trace(go.Scatter(name = 'Confidence Interval',
                             x = pd.period_range(start = data_tmp['Births'].index[-1], end = None, periods = 16, freq = 'D').to_timestamp(),
                             y = np.concatenate([np.array(data.iloc[-start_num:, :]).flatten()[29].reshape(-1), pred_min_interval]),
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             mode = 'lines',
                             fillcolor = 'rgba(179,226,205,0.6)',
                             fill = 'tonexty',
                             showlegend = True), row=1,col=1)
    ## mean value
    fig.add_trace(go.Scatter(x = pd.period_range(start = data_tmp['Births'].index[0], end = None, periods = 45, freq = 'D').to_timestamp(),
                             y = np.array(data.iloc[-start_num:, :]).flatten(), 
                             name = "Real value", line=dict(color="#636EFA")), row=1,col=1)
    fig.add_trace(go.Scatter(x = pd.period_range(start = data_tmp['Births'].index[-1], end = None, periods = 16, freq = 'D').to_timestamp(), 
                             y = np.concatenate([np.array(data.iloc[-start_num:, :]).flatten()[29].reshape(-1), pred_mean]), 
                             name = "Prediction average", line=dict(color="red")), row=1,col=1)
            
    # plot(1,2) - trend
    ## CI
    fig.add_trace(go.Scatter(name = 'Lower Bound',
                             x = pd.period_range(start = data_tmp['Births'].index[-1], end = None, periods = 16, freq = 'D').to_timestamp()[1:],
                             y = trend_max_interval,
                             mode = 'lines',
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             showlegend = False), row=1,col=2)
    fig.add_trace(go.Scatter(name = 'Upper Bound',
                             x = pd.period_range(start = data_tmp['Births'].index[-1], end = None, periods = 16, freq = 'D').to_timestamp()[1:],
                             y = trend_min_interval,
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             mode = 'lines',
                             fillcolor = 'rgba(179,226,205,0.6)',
                             fill = 'tonexty',
                             showlegend = False), row=1,col=2)
    ## mean value
    fig.add_trace(go.Scatter(x = pd.period_range(start = data_tmp['Births'].index[-1], end = None, periods = 16, freq = 'D').to_timestamp()[1:], 
                             y = trend_mean, name = "Trend average", line=dict(color="red"), showlegend = False), row=1,col=2)
            
    # plot(2,2) - seasonality
    ## CI
    fig.add_trace(go.Scatter(name = 'Lower Bound',
                             x = pd.period_range(start = data_tmp['Births'].index[-1], end = None, periods = 16, freq = 'D').to_timestamp()[1:],
                             y = seasonal_max_interval,
                             mode = 'lines',
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             showlegend = False), row=2,col=2)
    fig.add_trace(go.Scatter(name = 'Upper Bound',
                             x = pd.period_range(start = data_tmp['Births'].index[-1], end = None, periods = 16, freq = 'D').to_timestamp()[1:],
                             y = seasonal_min_interval,
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             mode = 'lines',
                             fillcolor = 'rgba(179,226,205,0.6)',
                             fill = 'tonexty',
                             showlegend = False), row=2,col=2)
    ## mean value
    fig.add_trace(go.Scatter(x = pd.period_range(start = data_tmp['Births'].index[-1], end = None, periods = 16, freq = 'D').to_timestamp()[1:], 
                             y = seasonal_mean, name = "Seasonality average", line=dict(color="red"), showlegend = False), row=2,col=2)
    # dash line
    full_fig = fig.full_figure_for_development()
    fig.add_shape(type="line", xref='x', yref='paper',
                  x0=list(data_tmp['Births'].index)[-1], y0 = full_fig.layout.yaxis.range[0],
                  x1=list(data_tmp['Births'].index)[-1], y1 = full_fig.layout.yaxis.range[1],
                  line=dict(color="black", width=1, dash="dash"),row=1,col=1)

    fig.update_layout(height=600, width=1300, title_text="Daily Female Births Prediction")

    # delete y axis in Trend & Seasonality
    fig.update_yaxes(showticklabels=False, row=1,col=2)
    fig.update_yaxes(showticklabels=False, row=2,col=2)

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True) # 테두리
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True) # 테두리
    pio.write_image(fig, f'/content/drive/MyDrive/2022_confer_image/Bench/Birth/nbeats_bench_prediction{start_num}.png', engine='kaleido')
        
    fig.show()

In [ ]:
for start_num in list(range(45, round(len(data)*2/10)-1)):
    data_tmp = data.iloc[-start_num:][:30]
    tmp = yscaler.transform(np.array(data_tmp['Births']).reshape(-1, 1))
    data_tmp['Births'] = tmp
    Y = data_tmp.iloc[:, 0].values
    Y = torch.FloatTensor(Y)
    Y = Y.unsqueeze(-1)
    Y = Y.unsqueeze(0) # batch_size
    y_pred, trend, seasonality = main(50,15,yscaler)
    plot_prediction(y_pred, trend, seasonality, start_num)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



...

/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



# Sunspot Data set

## Data Preprocessing

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/2022 컨퍼/benchmark_data/benchmark_Sunspots.csv', header=0)
data = data[['Date', 'Sunspots']]
data.set_index('Date', inplace=True)
data2 = data.copy()
data2

,Sunspots
Date,
1749-01,58.0
1749-02,62.6
1749-03,70.0
1749-04,55.7
1749-05,85.0
...,...
1983-08,71.8
1983-09,50.3
1983-10,55.8


## Load Scaler & Model

In [ ]:
window_size = 60
yscaler2 = pickle.load(open('/content/drive/Shareddrives/[투빅스] 16기 & 17기 시계열 컨퍼런스/예측/Nbeats_benchmark/Sunspots/scaler.pkl', 'rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning:

Trying to unpickle estimator StandardScaler from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations



In [ ]:
model2 = NBeatsNet(stack_types=(NBeatsNet.TREND_BLOCK, NBeatsNet.SEASONALITY_BLOCK),
                    forecast_length=30,
                    backcast_length=60,
                    hidden_layer_units=128,
                    )
model2.load_state_dict(torch.load('/content/drive/Shareddrives/[투빅스] 16기 & 17기 시계열 컨퍼런스/예측/Nbeats_benchmark/Sunspots/model.pth', map_location='cpu'))

| N-Beats
| --  Stack Trend (#0) (share_weights_in_stack=False)
     | -- TrendBlock(units=128, thetas_dim=4, backcast_length=60, forecast_length=30, share_thetas=True) at @140655456821584
     | -- TrendBlock(units=128, thetas_dim=4, backcast_length=60, forecast_length=30, share_thetas=True) at @140655444359824
     | -- TrendBlock(units=128, thetas_dim=4, backcast_length=60, forecast_length=30, share_thetas=True) at @140655443222288
| --  Stack Seasonality (#1) (share_weights_in_stack=False)
     | -- SeasonalityBlock(units=128, thetas_dim=30, backcast_length=60, forecast_length=30, share_thetas=True) at @140655443218704
     | -- SeasonalityBlock(units=128, thetas_dim=30, backcast_length=60, forecast_length=30, share_thetas=True) at @140655457504592
     | -- SeasonalityBlock(units=128, thetas_dim=30, backcast_length=60, forecast_length=30, share_thetas=True) at @140655448782032


<All keys matched successfully>

### Predict

In [ ]:
def main2(sample_size, prediction_size, yscaler):
    y_pred = np.zeros((sample_size, prediction_size))
    trend = np.zeros((sample_size, prediction_size))
    seasonality = np.zeros((sample_size, prediction_size))
    for i in range(sample_size):
        backcast, pred, trend_forecast, seasonal_forecast = model2(Y)
        pred = pred.detach().cpu().numpy()
        trend_forecast = trend_forecast.detach().cpu().numpy()
        seasonal_forecast = seasonal_forecast.detach().cpu().numpy()

        y_pred[i,:] = yscaler.inverse_transform(pred)
        trend[i,:] = yscaler.inverse_transform(trend_forecast)
        seasonality[i,:] = y_pred[i,:] - trend[i,:]
    return y_pred, trend, seasonality

In [ ]:
def plot_prediction2(y_pred, trend, seasonality, start_num):
    fig = make_subplots(
    subplot_titles=['True Vs Predicted','Trend','Seasonality'],
    rows=2, cols=2,
    vertical_spacing=0.15,
    horizontal_spacing=0.025,
    column_widths=[0.9, 0.6],
    row_heights=[0.8, 0.8],
    specs=[[{"rowspan": 2}, {}], [None, {}]])
            
    # confidence interval
    pred_mean, pred_min_interval, pred_max_interval = confidence_interval(y_pred, 0.01)
    trend_mean, trend_min_interval, trend_max_interval = confidence_interval(trend, 0.01)
    seasonal_mean, seasonal_min_interval, seasonal_max_interval = confidence_interval(seasonality, 0.01)
            
    # plot(1,1) - prediction vs real_value
    ## CI
    fig.add_trace(go.Scatter(name = 'Lower Bound',
                             x = pd.period_range(start = data_tmp['Sunspots'].index[-1], end = None, periods = 31, freq = 'M').to_timestamp(),
                             y = np.concatenate([np.array(data.iloc[-start_num:, :]).flatten()[59].reshape(-1), pred_max_interval]),
                             mode = 'lines',
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             showlegend = False), row=1,col=1)
    fig.add_trace(go.Scatter(name = 'Confidence Interval',
                             x = pd.period_range(start = data_tmp['Sunspots'].index[-1], end = None, periods = 31, freq = 'M').to_timestamp(),
                             y = np.concatenate([np.array(data.iloc[-start_num:, :]).flatten()[59].reshape(-1), pred_min_interval]),
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             mode = 'lines',
                             fillcolor = 'rgba(179,226,205,0.6)',
                             fill = 'tonexty',
                             showlegend = True), row=1,col=1)
    ## mean value
    fig.add_trace(go.Scatter(x = pd.period_range(start = data_tmp['Sunspots'].index[0], end = None, periods = 90, freq = 'M').to_timestamp(),
                             y = np.array(data.iloc[-start_num:, :]).flatten(), 
                             name = "Real value", line=dict(color="#636EFA")), row=1,col=1)
    fig.add_trace(go.Scatter(x = pd.period_range(start = data_tmp['Sunspots'].index[-1], end = None, periods = 31, freq = 'M').to_timestamp(), 
                             y = np.concatenate([np.array(data.iloc[-start_num:, :]).flatten()[59].reshape(-1), pred_mean]), 
                             name = "Prediction average", line=dict(color="red")), row=1,col=1)
            
    # plot(1,2) - trend
    ## CI
    fig.add_trace(go.Scatter(name = 'Lower Bound',
                             x = pd.period_range(start = data_tmp['Sunspots'].index[-1], end = None, periods = 31, freq = 'M').to_timestamp()[1:],
                             y = trend_max_interval,
                             mode = 'lines',
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             showlegend = False), row=1,col=2)
    fig.add_trace(go.Scatter(name = 'Upper Bound',
                             x = pd.period_range(start = data_tmp['Sunspots'].index[-1], end = None, periods = 31, freq = 'M').to_timestamp()[1:],
                             y = trend_min_interval,
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             mode = 'lines',
                             fillcolor = 'rgba(179,226,205,0.6)',
                             fill = 'tonexty',
                             showlegend = False), row=1,col=2)
    ## mean value
    fig.add_trace(go.Scatter(x = pd.period_range(start = data_tmp['Sunspots'].index[-1], end = None, periods = 31, freq = 'M').to_timestamp()[1:], 
                             y = trend_mean, name = "Trend average", line=dict(color="red"), showlegend = False), row=1,col=2)
            
    # plot(2,2) - seasonality
    ## CI
    fig.add_trace(go.Scatter(name = 'Lower Bound',
                             x = pd.period_range(start = data_tmp['Sunspots'].index[-1], end = None, periods = 31, freq = 'M').to_timestamp()[1:],
                             y = seasonal_max_interval,
                             mode = 'lines',
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             showlegend = False), row=2,col=2)
    fig.add_trace(go.Scatter(name = 'Upper Bound',
                             x = pd.period_range(start = data_tmp['Sunspots'].index[-1], end = None, periods = 31, freq = 'M').to_timestamp()[1:],
                             y = seasonal_min_interval,
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             mode = 'lines',
                             fillcolor = 'rgba(179,226,205,0.6)',
                             fill = 'tonexty',
                             showlegend = False), row=2,col=2)
    ## mean value
    fig.add_trace(go.Scatter(x = pd.period_range(start = data_tmp['Sunspots'].index[-1], end = None, periods = 31, freq = 'M').to_timestamp()[1:], 
                             y = seasonal_mean, name = "Seasonality average", line=dict(color="red"), showlegend = False), row=2,col=2)
    # dash line
    full_fig = fig.full_figure_for_development()
    fig.add_shape(type="line", xref='x', yref='paper',
                  x0=list(data_tmp['Sunspots'].index)[-1], y0 = full_fig.layout.yaxis.range[0],
                  x1=list(data_tmp['Sunspots'].index)[-1], y1 = full_fig.layout.yaxis.range[1],
                  line=dict(color="black", width=1, dash="dash"),row=1,col=1)

    fig.update_layout(height=600, width=1300, title_text="Monthly Sunspots Prediction")

    # delete y axis in Trend & Seasonality
    fig.update_yaxes(showticklabels=False, row=1,col=2)
    fig.update_yaxes(showticklabels=False, row=2,col=2)

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True) # 테두리
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True) # 테두리
    pio.write_image(fig, f'/content/drive/MyDrive/2022_confer_image/Bench/Sunspot/nbeats_bench_prediction{start_num}.png', engine='kaleido')
        
    fig.show()

In [ ]:
for start_num in list(range(90, round(len(data)*2/10)-1, 80)):
    data_tmp = data2.iloc[-start_num:][:60]
    tmp = yscaler2.transform(np.array(data_tmp['Sunspots']).reshape(-1, 1))
    data_tmp['Sunspots'] = tmp
    Y = data_tmp.iloc[:, 0].values
    Y = torch.FloatTensor(Y)
    Y = Y.unsqueeze(-1)
    Y = Y.unsqueeze(0) # batch_size
    y_pred, trend, seasonality = main2(50,30,yscaler2)
    plot_prediction2(y_pred, trend, seasonality, start_num)

/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



# Temperature Data set

## Data Preprocessing

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/2022 컨퍼/benchmark_data/benchmark_Temp.csv', header=0)
data = data[['Date', 'Temp']]
data.set_index('Date', inplace=True)
data2 = data.copy()
data2

,Temp
Date,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8
...,...
1990-12-27,14.0
1990-12-28,13.6
1990-12-29,13.5


## Load Scaler & Model

In [ ]:
window_size = 70
yscaler3 = pickle.load(open('/content/drive/Shareddrives/[투빅스] 16기 & 17기 시계열 컨퍼런스/예측/Nbeats_benchmark/Temp/scaler.pkl', 'rb'))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning:

Trying to unpickle estimator StandardScaler from version 1.1.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations



In [ ]:
model3 = NBeatsNet(stack_types=(NBeatsNet.TREND_BLOCK, NBeatsNet.SEASONALITY_BLOCK),
                    forecast_length=35,
                    backcast_length=70,
                    hidden_layer_units=128,
                    )
model3.load_state_dict(torch.load('/content/drive/Shareddrives/[투빅스] 16기 & 17기 시계열 컨퍼런스/예측/Nbeats_benchmark/Temp/model.pth', map_location='cpu'))

| N-Beats
| --  Stack Trend (#0) (share_weights_in_stack=False)
     | -- TrendBlock(units=128, thetas_dim=4, backcast_length=70, forecast_length=35, share_thetas=True) at @140655444999184
     | -- TrendBlock(units=128, thetas_dim=4, backcast_length=70, forecast_length=35, share_thetas=True) at @140655444999632
     | -- TrendBlock(units=128, thetas_dim=4, backcast_length=70, forecast_length=35, share_thetas=True) at @140655444963856
| --  Stack Seasonality (#1) (share_weights_in_stack=False)
     | -- SeasonalityBlock(units=128, thetas_dim=35, backcast_length=70, forecast_length=35, share_thetas=True) at @140655444999440
     | -- SeasonalityBlock(units=128, thetas_dim=35, backcast_length=70, forecast_length=35, share_thetas=True) at @140655447178768
     | -- SeasonalityBlock(units=128, thetas_dim=35, backcast_length=70, forecast_length=35, share_thetas=True) at @140655456823824


<All keys matched successfully>

### Predict

In [ ]:
def main3(sample_size, prediction_size, yscaler):
    y_pred = np.zeros((sample_size, prediction_size))
    trend = np.zeros((sample_size, prediction_size))
    seasonality = np.zeros((sample_size, prediction_size))
    for i in range(sample_size):
        backcast, pred, trend_forecast, seasonal_forecast = model3(Y)
        pred = pred.detach().cpu().numpy()
        trend_forecast = trend_forecast.detach().cpu().numpy()
        seasonal_forecast = seasonal_forecast.detach().cpu().numpy()

        y_pred[i,:] = yscaler.inverse_transform(pred)
        trend[i,:] = yscaler.inverse_transform(trend_forecast)
        seasonality[i,:] = y_pred[i,:] - trend[i,:]
    return y_pred, trend, seasonality

In [ ]:
def plot_prediction3(y_pred, trend, seasonality, start_num):
    fig = make_subplots(
    subplot_titles=['True Vs Predicted','Trend','Seasonality'],
    rows=2, cols=2,
    vertical_spacing=0.15,
    horizontal_spacing=0.025,
    column_widths=[0.9, 0.6],
    row_heights=[0.8, 0.8],
    specs=[[{"rowspan": 2}, {}], [None, {}]])
            
    # confidence interval
    pred_mean, pred_min_interval, pred_max_interval = confidence_interval(y_pred, 0.01)
    trend_mean, trend_min_interval, trend_max_interval = confidence_interval(trend, 0.01)
    seasonal_mean, seasonal_min_interval, seasonal_max_interval = confidence_interval(seasonality, 0.01)

    # plot(1,1) - prediction vs real_value
    ## CI
    fig.add_trace(go.Scatter(name = 'Lower Bound',
                             x = pd.period_range(start = data_tmp['Temp'].index[-1], end = None, periods = 36, freq = 'D').to_timestamp(),
                             y = np.concatenate([np.array(data.iloc[-(start_num):, :]).flatten()[69].reshape(-1), pred_max_interval]),
                             mode = 'lines',
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             showlegend = False), row=1,col=1)
    fig.add_trace(go.Scatter(name = 'Confidence Interval',
                             x = pd.period_range(start = data_tmp['Temp'].index[-1], end = None, periods = 36, freq = 'D').to_timestamp(),
                             y = np.concatenate([np.array(data.iloc[-(start_num):, :]).flatten()[69].reshape(-1), pred_min_interval]),
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             mode = 'lines',
                             fillcolor = 'rgba(179,226,205,0.6)',
                             fill = 'tonexty',
                             showlegend = True), row=1,col=1)
    ## mean value
    fig.add_trace(go.Scatter(x = pd.period_range(start = data_tmp['Temp'].index[0], end = None, periods = 105, freq = 'D').to_timestamp(),
                             y = np.array(data.iloc[-(start_num):, :]).flatten(), 
                             name = "Real value", line=dict(color="#636EFA")), row=1,col=1)
    fig.add_trace(go.Scatter(x = pd.period_range(start = data_tmp['Temp'].index[-1], end = None, periods = 36, freq = 'D').to_timestamp(), 
                             y = np.concatenate([np.array(data.iloc[-(start_num):, :]).flatten()[69].reshape(-1), pred_mean]), 
                             name = "Prediction average", line=dict(color="red")), row=1,col=1)
            
    # plot(1,2) - trend
    ## CI
    fig.add_trace(go.Scatter(name = 'Lower Bound',
                             x = pd.period_range(start = data_tmp['Temp'].index[-1], end = None, periods = 36, freq = 'D').to_timestamp()[1:],
                             y = trend_max_interval,
                            mode = 'lines',
                            marker = dict(color="rgb(179,226,205)"),
                            line = dict(width=0),
                            showlegend = False), row=1,col=2)
    fig.add_trace(go.Scatter(name = 'Upper Bound',
                             x = pd.period_range(start = data_tmp['Temp'].index[-1], end = None, periods = 36, freq = 'D').to_timestamp()[1:],
                             y = trend_min_interval,
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             mode = 'lines',
                             fillcolor = 'rgba(179,226,205,0.6)',
                             fill = 'tonexty',
                             showlegend = False), row=1,col=2)
    ## mean value
    fig.add_trace(go.Scatter(x = pd.period_range(start = data_tmp['Temp'].index[-1], end = None, periods = 36, freq = 'D').to_timestamp()[1:], 
                             y = trend_mean, name = "Trend average", line=dict(color="red"), showlegend = False), row=1,col=2)
            
    # plot(2,2) - seasonality
    ## CI
    fig.add_trace(go.Scatter(name = 'Lower Bound',
                             x = pd.period_range(start = data_tmp['Temp'].index[-1], end = None, periods = 36, freq = 'D').to_timestamp()[1:],
                             y = seasonal_max_interval,
                             mode = 'lines',
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             showlegend = False), row=2,col=2)
    fig.add_trace(go.Scatter(name = 'Upper Bound',
                             x = pd.period_range(start = data_tmp['Temp'].index[-1], end = None, periods = 36, freq = 'D').to_timestamp()[1:],
                             y = seasonal_min_interval,
                             marker = dict(color="rgb(179,226,205)"),
                             line = dict(width=0),
                             mode = 'lines',
                             fillcolor = 'rgba(179,226,205,0.6)',
                             fill = 'tonexty',
                             showlegend = False), row=2,col=2)
    ## mean value
    fig.add_trace(go.Scatter(x = pd.period_range(start = data_tmp['Temp'].index[-1], end = None, periods = 36, freq = 'D').to_timestamp()[1:], 
                             y = seasonal_mean, name = "Seasonality average", line=dict(color="red"), showlegend = False), row=2,col=2)
    # dash line
    full_fig = fig.full_figure_for_development()
    fig.add_shape(type="line", xref='x', yref='paper',
                  x0=list(data_tmp['Temp'].index)[-1], y0 = full_fig.layout.yaxis.range[0],
                  x1=list(data_tmp['Temp'].index)[-1], y1 = full_fig.layout.yaxis.range[1],
                  line=dict(color="black", width=1, dash="dash"),row=1,col=1)

    fig.update_layout(height=600, width=1300, title_text="Minimum Daily Temperature Prediction")

    # delete y axis in Trend & Seasonality
    fig.update_yaxes(showticklabels=False, row=1,col=2)
    fig.update_yaxes(showticklabels=False, row=2,col=2)

    fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True) # 테두리
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True) # 테두리
    pio.write_image(fig, f'/content/drive/MyDrive/2022_confer_image/Bench/Temperature/nbeats_bench_prediction{start_num}.png', engine='kaleido')
        
    fig.show()

In [ ]:
for start_num in list(range(105, round(len(data)*2/10)-1, 60)):
    data_tmp = data2.iloc[-start_num:][:70]
    tmp = yscaler3.transform(np.array(data_tmp['Temp']).reshape(-1, 1))
    data_tmp['Temp'] = tmp
    Y = data_tmp.iloc[:, 0].values
    Y = torch.FloatTensor(Y)
    Y = Y.unsqueeze(-1)
    Y = Y.unsqueeze(0) # batch_size
    y_pred, trend, seasonality = main3(50,35,yscaler3)
    plot_prediction3(y_pred, trend, seasonality, start_num)

/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



...

/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False



/usr/local/lib/python3.7/dist-packages/plotly/io/_kaleido.py:341: UserWarning:

full_figure_for_development is not recommended or necessary for production use in most circumstances. 
To suppress this warning, set warn=False

